<img src="https://relevance.ai/wp-content/uploads/2021/11/logo.79f303e-1.svg" width="150" alt="Relevance AI" />
<h5> Developer-first vector platform for ML teams </h5>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RelevanceAI/workflows/blob/main/workflows/taxonomy/Taxonomy.ipynb)

Inserting your own taxonomy or labels with Relevance AI allows users to better understand.

In [ ]:
# @title After filling this form, press the top left button.
# You can grab your token here https://cloud.relevance.ai/sdk/api

token = "<copy paste from https://cloud.relevance.ai/sdk/api>"#@param  {type:"string"}
dataset_id = "<your dataset ID here>" #@param {type:"string"}
taxonomy_labels = "dairy products, bread"  #@param {type:"string"}
field = "text" #@param  {type:"string"} Could be a vector field or a text field

#@markdown Optional Configurations
max_number_of_labels = 1 #@param  {type:"integer"}
model_id = "sentence-transformers/all-mpnet-base-v2" #@param  {type:"string"}


def strip_empty_string(list):
  without_empty_strings = []
  for string in list:
    if (string != ''):
        without_empty_strings.append(string.strip())
  return without_empty_strings

taxonomy_labels = strip_empty_string(taxonomy_labels.split(','))

!pip install -q RelevanceAI==2.3.2
!pip install -q sentence-transformers==2.2.0

from relevanceai import Client

client = Client(token=token)
ds = client.Dataset(dataset_id)
from sentence_transformers import SentenceTransformer

enc = SentenceTransformer(model_id)

label_documents = [
    {"label": l, "label_vector_": enc.encode(l).tolist()} for l in taxonomy_labels
]
if not field.endswith("_vector_"):
    ds.vectorize_text(fields=[field], models=[model_id])

if not field.endswith("_vector_"):
    # rename to a different vector
    field = field + "_" + model_id + "_vector_"
vector_field = field.replace("/", "_")
ds.label(
    vector_fields=[vector_field],
    label_documents=label_documents,
    max_number_of_labels=max_number_of_labels,
)